# Tree-based Analyses

## Put an Intro here

In [1]:
from sklearn.ensemble import AdaBoostClassifier 
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split
import pandas as pd

#import mapped version of data set here
readmit = pd.read_csv('diabetes_readmission_onehot.csv') 

In [2]:
# capture independent variables in list
features = list(readmit)
features.remove('readmit30')

In [3]:
# split the data into a training and test set
Xtrain, Xtest, Ytrain, Ytest = train_test_split(readmit[features].values, 
                                              (readmit.readmit30 == 1).values, test_size = .2, random_state = 7)

In [4]:
# build model on training data
dt = DecisionTreeClassifier() 
clf = AdaBoostClassifier(base_estimator = dt, n_estimators = 50, learning_rate = 1, random_state = 7)
model = clf.fit(Xtrain, Ytrain)

# check performance (model accuracy) on test data
model.score(Xtest, Ytest)

0.85182137781434664

In [ ]:
# results summary (won't be a tree b/c boosting used -- explain?)